# AgentCore Resource Cleanup

This notebook cleans up AWS resources created by the Agentic-Metrics-AgentCore.ipynb notebook.

## Resources to Clean Up:
- AgentCore Runtime (citysearch agent)
- ECR Repository (bedrock-agentcore-citysearch)
- IAM Roles (AmazonBedrockAgentCoreSDKRuntime and AmazonBedrockAgentCoreSDKCodeBuild)
- CodeBuild Project (bedrock-agentcore-citysearch-builder)
- S3 Objects (source.zip uploads)
- CloudWatch Log Groups

**⚠️ Warning: This will permanently delete all AgentCore resources. Make sure you no longer need them.**

## 1. Setup and Account ID Masking

In [ ]:
import boto3
import re
import builtins
import json
from botocore.exceptions import ClientError

# Account ID masking for security
def mask_account_id(text):
    return re.sub(r'\b\d{12}\b', 'XXXXXXXXXXXX', str(text))

_original_print = builtins.print

def safe_print(*args, **kwargs):
    args = [mask_account_id(arg) for arg in args]
    _original_print(*args, **kwargs)

builtins.print = safe_print

# Initialize AWS clients
session = boto3.Session()
region = session.region_name or 'us-east-1'
account_id = boto3.client('sts').get_caller_identity()['Account']

agentcore_client = boto3.client('bedrock-agentcore', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
iam_client = boto3.client('iam')
codebuild_client = boto3.client('codebuild', region_name=region)
s3_client = boto3.client('s3', region_name=region)
logs_client = boto3.client('logs', region_name=region)

print(f"✅ Initialized cleanup for region: {region}")
print(f"✅ Account ID: {mask_account_id(account_id)}")

## 2. List Current AgentCore Resources

In [ ]:
# Configuration - Update these if you used different names
AGENT_NAME = "citysearch"
ECR_REPO_NAME = f"bedrock-agentcore-{AGENT_NAME}"
CODEBUILD_PROJECT_NAME = f"bedrock-agentcore-{AGENT_NAME}-builder"
RUNTIME_ROLE_PREFIX = "AmazonBedrockAgentCoreSDKRuntime"
CODEBUILD_ROLE_PREFIX = "AmazonBedrockAgentCoreSDKCodeBuild"

print(f"🔍 Looking for resources with agent name: {AGENT_NAME}")
print(f"🔍 ECR Repository: {ECR_REPO_NAME}")
print(f"🔍 CodeBuild Project: {CODEBUILD_PROJECT_NAME}")

In [ ]:
# List AgentCore runtimes
try:
    response = agentcore_client.list_runtimes()
    runtimes = response.get('runtimes', [])
    
    citysearch_runtimes = [r for r in runtimes if AGENT_NAME in r.get('name', '').lower()]
    
    if citysearch_runtimes:
        print(f"📋 Found {len(citysearch_runtimes)} AgentCore runtime(s):")
        for runtime in citysearch_runtimes:
            print(f"  • Name: {runtime.get('name')}")
            print(f"  • ARN: {mask_account_id(runtime.get('arn', 'N/A'))}")
            print(f"  • Status: {runtime.get('status', 'N/A')}")
    else:
        print(f"ℹ️  No AgentCore runtimes found for agent: {AGENT_NAME}")
        
except ClientError as e:
    print(f"❌ Error listing AgentCore runtimes: {e}")

## 3. Delete AgentCore Runtime

In [ ]:
# Delete AgentCore runtimes
deleted_runtimes = []

if citysearch_runtimes:
    for runtime in citysearch_runtimes:
        runtime_arn = runtime.get('arn')
        runtime_name = runtime.get('name')
        
        try:
            print(f"🗑️  Deleting AgentCore runtime: {runtime_name}")
            agentcore_client.delete_runtime(runtimeArn=runtime_arn)
            deleted_runtimes.append(runtime_name)
            print(f"✅ Successfully deleted runtime: {runtime_name}")
            
        except ClientError as e:
            if e.response['Error']['Code'] == 'ResourceNotFoundException':
                print(f"ℹ️  Runtime {runtime_name} not found (may already be deleted)")
            else:
                print(f"❌ Error deleting runtime {runtime_name}: {e}")
else:
    print("ℹ️  No AgentCore runtimes to delete")

## 4. Delete ECR Repository

In [ ]:
# Delete ECR repository
try:
    print(f"🗑️  Deleting ECR repository: {ECR_REPO_NAME}")
    ecr_client.delete_repository(
        repositoryName=ECR_REPO_NAME,
        force=True  # Delete even if it contains images
    )
    print(f"✅ Successfully deleted ECR repository: {ECR_REPO_NAME}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'RepositoryNotFoundException':
        print(f"ℹ️  ECR repository {ECR_REPO_NAME} not found (may already be deleted)")
    else:
        print(f"❌ Error deleting ECR repository: {e}")

## 5. Delete CodeBuild Project

In [ ]:
# Delete CodeBuild project
try:
    print(f"🗑️  Deleting CodeBuild project: {CODEBUILD_PROJECT_NAME}")
    codebuild_client.delete_project(name=CODEBUILD_PROJECT_NAME)
    print(f"✅ Successfully deleted CodeBuild project: {CODEBUILD_PROJECT_NAME}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceNotFoundException':
        print(f"ℹ️  CodeBuild project {CODEBUILD_PROJECT_NAME} not found (may already be deleted)")
    else:
        print(f"❌ Error deleting CodeBuild project: {e}")

## 6. Clean Up IAM Roles

In [ ]:
# List and delete IAM roles created by AgentCore
def delete_iam_role_with_policies(role_name):
    try:
        # Detach managed policies
        attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)
        for policy in attached_policies['AttachedPolicies']:
            iam_client.detach_role_policy(
                RoleName=role_name,
                PolicyArn=policy['PolicyArn']
            )
            print(f"  • Detached policy: {policy['PolicyName']}")
        
        # Delete inline policies
        inline_policies = iam_client.list_role_policies(RoleName=role_name)
        for policy_name in inline_policies['PolicyNames']:
            iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
            print(f"  • Deleted inline policy: {policy_name}")
        
        # Delete the role
        iam_client.delete_role(RoleName=role_name)
        print(f"✅ Successfully deleted IAM role: {role_name}")
        return True
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print(f"ℹ️  IAM role {role_name} not found (may already be deleted)")
        else:
            print(f"❌ Error deleting IAM role {role_name}: {e}")
        return False

# Find and delete AgentCore-related roles
try:
    roles = iam_client.list_roles()['Roles']
    agentcore_roles = [
        role for role in roles 
        if (RUNTIME_ROLE_PREFIX in role['RoleName'] or 
            CODEBUILD_ROLE_PREFIX in role['RoleName']) and 
            region in role['RoleName']
    ]
    
    if agentcore_roles:
        print(f"📋 Found {len(agentcore_roles)} AgentCore IAM role(s):")
        for role in agentcore_roles:
            role_name = role['RoleName']
            print(f"🗑️  Deleting IAM role: {role_name}")
            delete_iam_role_with_policies(role_name)
    else:
        print("ℹ️  No AgentCore IAM roles found")
        
except ClientError as e:
    print(f"❌ Error listing IAM roles: {e}")

## 7. Clean Up S3 Objects

In [ ]:
# Clean up S3 objects (source.zip uploads)
# Note: AgentCore typically uses a bucket named after the account/region
try:
    # List buckets to find potential AgentCore buckets
    buckets = s3_client.list_buckets()['Buckets']
    agentcore_buckets = [
        bucket for bucket in buckets 
        if 'agentcore' in bucket['Name'].lower() or 
           'bedrock' in bucket['Name'].lower()
    ]
    
    for bucket in agentcore_buckets:
        bucket_name = bucket['Name']
        try:
            # List objects with citysearch prefix
            objects = s3_client.list_objects_v2(
                Bucket=bucket_name,
                Prefix=f"{AGENT_NAME}/"
            )
            
            if 'Contents' in objects:
                print(f"🗑️  Cleaning up S3 objects in bucket: {bucket_name}")
                for obj in objects['Contents']:
                    s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
                    print(f"  • Deleted: {obj['Key']}")
                print(f"✅ Cleaned up S3 objects for {AGENT_NAME}")
            else:
                print(f"ℹ️  No S3 objects found for {AGENT_NAME} in bucket: {bucket_name}")
                
        except ClientError as e:
            if e.response['Error']['Code'] == 'NoSuchBucket':
                print(f"ℹ️  Bucket {bucket_name} not found")
            else:
                print(f"❌ Error cleaning S3 bucket {bucket_name}: {e}")
    
    if not agentcore_buckets:
        print("ℹ️  No potential AgentCore S3 buckets found")
        
except ClientError as e:
    print(f"❌ Error listing S3 buckets: {e}")

## 8. Clean Up CloudWatch Log Groups

In [ ]:
# Clean up CloudWatch log groups
try:
    # Find log groups related to the agent
    log_groups = logs_client.describe_log_groups(
        logGroupNamePrefix='/aws/bedrock-agentcore/runtimes/'
    )['logGroups']
    
    citysearch_log_groups = [
        lg for lg in log_groups 
        if AGENT_NAME in lg['logGroupName'].lower()
    ]
    
    if citysearch_log_groups:
        print(f"📋 Found {len(citysearch_log_groups)} CloudWatch log group(s):")
        for log_group in citysearch_log_groups:
            log_group_name = log_group['logGroupName']
            try:
                print(f"🗑️  Deleting log group: {log_group_name}")
                logs_client.delete_log_group(logGroupName=log_group_name)
                print(f"✅ Successfully deleted log group: {log_group_name}")
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print(f"ℹ️  Log group {log_group_name} not found")
                else:
                    print(f"❌ Error deleting log group {log_group_name}: {e}")
    else:
        print(f"ℹ️  No CloudWatch log groups found for agent: {AGENT_NAME}")
        
except ClientError as e:
    print(f"❌ Error listing CloudWatch log groups: {e}")

## 9. Clean Up Notebook Outputs (Remove Account IDs)

In [1]:
import json
import os
import re

def clean_notebook_outputs(notebook_path):
    """Remove account IDs from notebook cell outputs"""
    try:
        with open(notebook_path, 'r', encoding='utf-8') as f:
            notebook = json.load(f)
        
        cleaned_cells = 0
        
        for cell in notebook.get('cells', []):
            if cell.get('cell_type') == 'code' and 'outputs' in cell:
                original_outputs = json.dumps(cell['outputs'])
                
                # Clean outputs recursively
                def clean_data(data):
                    if isinstance(data, dict):
                        return {k: clean_data(v) for k, v in data.items()}
                    elif isinstance(data, list):
                        return [clean_data(item) for item in data]
                    elif isinstance(data, str):
                        # Replace 12-digit account IDs with XXXXXXXXXXXX
                        return re.sub(r'\b\d{12}\b', 'XXXXXXXXXXXX', data)
                    else:
                        return data
                
                cell['outputs'] = clean_data(cell['outputs'])
                
                # Check if anything was changed
                if original_outputs != json.dumps(cell['outputs']):
                    cleaned_cells += 1
        
        # Write back the cleaned notebook
        with open(notebook_path, 'w', encoding='utf-8') as f:
            json.dump(notebook, f, indent=1, ensure_ascii=False)
        
        return cleaned_cells
        
    except Exception as e:
        print(f"❌ Error cleaning notebook {notebook_path}: {e}")
        return 0

# Clean the main notebooks
notebooks_to_clean = [
    'Agentic-Metrics-AgentCore.ipynb',
    'Agent-and-tool-evals-with-xray.ipynb'
]

print("🧹 Cleaning notebook outputs to remove account IDs...")
total_cleaned = 0

for notebook_name in notebooks_to_clean:
    if os.path.exists(notebook_name):
        cleaned = clean_notebook_outputs(notebook_name)
        total_cleaned += cleaned
        if cleaned > 0:
            print(f"✅ Cleaned {cleaned} cells in {notebook_name}")
        else:
            print(f"ℹ️  No account IDs found in {notebook_name}")
    else:
        print(f"⚠️  Notebook not found: {notebook_name}")

if total_cleaned > 0:
    print(f"\n✅ Successfully cleaned {total_cleaned} cells across all notebooks")
    print("📝 Notebooks are now safe to commit to GitHub")
else:
    print("\nℹ️  No account IDs found in notebook outputs")

🧹 Cleaning notebook outputs to remove account IDs...
✅ Cleaned 2 cells in Agentic-Metrics-AgentCore.ipynb
ℹ️  No account IDs found in Agent-and-tool-evals-with-xray.ipynb

✅ Successfully cleaned 2 cells across all notebooks
📝 Notebooks are now safe to commit to GitHub


## 10. Cleanup Summary

In [ ]:
print("\n" + "="*60)
print("🧹 CLEANUP SUMMARY")
print("="*60)

print(f"\n🤖 Agent: {AGENT_NAME}")
print(f"🌍 Region: {region}")
print(f"🔐 Account: {mask_account_id(account_id)}")

print("\n📋 Resources Processed:")
print(f"  • AgentCore Runtimes: {len(deleted_runtimes) if 'deleted_runtimes' in locals() else 0} deleted")
print(f"  • ECR Repository: {ECR_REPO_NAME}")
print(f"  • CodeBuild Project: {CODEBUILD_PROJECT_NAME}")
print(f"  • IAM Roles: Runtime and CodeBuild roles")
print(f"  • S3 Objects: Source uploads cleaned")
print(f"  • CloudWatch Logs: Agent log groups")
print(f"  • Notebook Outputs: Account IDs masked")

print("\n✅ Cleanup completed!")
print("\n💡 Next Steps:")
print("  1. Verify resources are deleted in AWS Console")
print("  2. Check for any remaining costs in AWS Cost Explorer")
print("  3. Notebooks are now safe to commit to GitHub")

print("\n⚠️  Note: Some resources may take a few minutes to fully delete")